In [ ]:
import os

import pandas as pd
import papermill as pm
from logger import logger

## Generate Embeddings

In [ ]:
notebook = "embeddings.ipynb"
pm.inspect_notebook(notebook)

In [ ]:
# model_name = "all-MiniLM-L6-v2"
# model_name = "all-mpnet-base-v2"
model_name = "bge-large-en-v1.5"
owner_name = "BAAI"

for strategy in ["mean"]:
    logger.info(
        f"Running {notebook} for {model_name} model with {strategy} pooling strategy"
    )
    pm.execute_notebook(
        input_path="embeddings.ipynb",
        output_path=os.path.join(
            "..",
            "artifacts",
            "notebooks",
            f"{model_name}_{strategy}_emb.ipynb",
        ),
        parameters={
            "MODEL_NAME": model_name,
            "POOLING_STRATEGY": strategy,
            "OWNER": owner_name,
        },
    )

## Evaluate Similarities

In [ ]:
notebook = "similarity.ipynb"
pm.inspect_notebook(notebook)

In [ ]:
# model_name = "all-MiniLM-L6-v2"
# model_name = "all-mpnet-base-v2"

for strategy in ["mean"]:
    logger.info(
        f"Running {notebook} for {model_name} model with {strategy} pooling strategy"
    )
    pm.execute_notebook(
        input_path="similarity.ipynb",
        output_path=os.path.join(
            "..",
            "artifacts",
            "notebooks",
            f"{model_name}_{strategy}_sim.ipynb",
        ),
        parameters={
            "MODEL_NAME": model_name,
            "POOLING_STRATEGY": strategy,
            "OWNER": owner_name,
        },
    )

### Clustering and Graph Visualisation

In [ ]:
notebook = "construct_graph.ipynb"
pm.inspect_notebook(notebook)

In [ ]:
directory = "../artifacts/outputs/"

input_file = f"{directory}/statistical_vector_based_embeddings_similarity_scores.xlsx"
excel = pd.ExcelFile(input_file)

for sheet_name in excel.sheet_names:
    df = pd.read_excel(input_file, sheet_name=sheet_name)
    output_file = f"{directory}/{sheet_name}_similarity_score.csv"
    df.to_csv(output_file, index=False)

In [ ]:
variation_list = [
    "mxbai-embed-large-v1_mean",
    "bge-large-en-v1.5-quant_mean",
    "bge-large-en-v1.5_mean",
    "d2v",
    "multi-qa-mpnet-base-cos-v1_mean_euclidean",
    "multi-qa-mpnet-base-dot-v1_cls_dot",
    "multi-qa-mpnet-base-cos-v1_mean_dot",
    "multi-qa-mpnet-base-cos-v1_mean_cos"
]

variation_list.extend(excel.sheet_names)
variation_list

In [ ]:
for em_model in variation_list:
    logger.info(f"Running {notebook} for {em_model} model")
    pm.execute_notebook(
        input_path=notebook,
        output_path=os.path.join(
            "..",
            "artifacts",
            "notebooks",
            f"{em_model}_construct_graph.ipynb",
        ),
        parameters={"EMBEDDING_MODEL": em_model},
    )

In [ ]:
notebook = "clusterviz.ipynb"
pm.inspect_notebook(notebook)

In [ ]:
for em_model in variation_list:
    logger.info(f"Running {notebook} for {em_model} model")
    pm.execute_notebook(
        input_path=notebook,
        output_path=os.path.join(
            "..",
            "artifacts",
            "notebooks",
            f"{em_model}_clusterviz.ipynb",
        ),
        parameters={"EMBEDDING_MODEL": em_model},
    )